In [3]:
import math
import torch
import torch.nn as nn
import numpy as np
from thop import profile
from einops import rearrange
from einops.layers.torch import Rearrange, Reduce
from timm.models.layers import trunc_normal_, DropPath

import pickle
import numpy as np
import urllib.request
import os.path
from pathlib import Path
from zipfile import ZipFile

import zipfile

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
pip install opencv-python

In [ ]:
pip install thop

In [ ]:
pip install timm

In [6]:
pip install einops

In [14]:
###Fonction pour telecharger BSD400 en zip
####Si elle ne parche pas, copier le lien et download le zip puis le poser dans le repertooire


def download_dataset(
    url='https://drive.google.com/file/d/1idKFDkAHJGAFDn1OyXZxsTbOSBx9GS8N/view?usp=sharing',
    filename='BSD400.zip'
    ):
    if os.path.isfile(filename):
        print(f'{filename} already exists. Skipping downloading.')
        return

    with urllib.request.urlopen(url) as downloaded_file, open(filename, 'wb') as f:
        f.write(downloaded_file.read())



In [15]:
##Extraction

def extract_datazip(
    path_src='BSD400.zip',
    path_dest='BSD400'
):
    with ZipFile(path_src, 'r') as zip_ref:
        zip_ref.extractall()
        return True

extract_datazip()

True

In [16]:
###On deplace les images dans un dossier
import os
import shutil

def move_files_to_train_folder():

  source_folder = './BSD400'
  destination_folder = './BDS400/real'

  if not os.path.exists(destination_folder):
      os.makedirs(destination_folder)

  for filename in os.listdir(source_folder):
      source_file = os.path.join(source_folder, filename)
      destination_file = os.path.join(destination_folder, filename)
      shutil.move(source_file, destination_file)

move_files_to_train_folder()

In [4]:
import cv2
def load_images(path, img_size=(128, 128)):
    images = []
    for filename in os.listdir(path):
        img = cv2.imread(os.path.join(path, filename))
        if img is not None:
            img = cv2.resize(img, img_size)
            img = img / 255.0  # Normalize to [0, 1]
            images.append(img)
    return np.array(images)

TRAIN_DIR = "./BDS400/real"
train_images = load_images(TRAIN_DIR)


In [5]:
def add_gaussian_noise(images, mean=0, sigma=0.1):
    noise = np.random.normal(mean, sigma, images.shape)
    noisy_images = np.clip(images + noise, 0, 1)
    return noisy_images

train_noisy_images = add_gaussian_noise(train_images)

In [6]:
len(train_images),len(train_noisy_images)

(400, 400)

In [19]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [20]:
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

train_dataloader = DataLoader(
    CustomDataset(train_noisy_images, train_images),
    batch_size=64,
    num_workers=4,
    shuffle=True)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
for noisy_images, clean_images in train_dataloader:
  print(noisy_images.shape)
  print(clean_images.shape)
  X_train = noisy_images.permute(0, 3, 1, 2)
  print(X_train.shape)
  break

torch.Size([64, 128, 128, 3])
torch.Size([64, 128, 128, 3])
torch.Size([64, 3, 128, 128])


## SCUNET

In [7]:
##fonction pour calcuer le psnr


def calculate_psnr(img1, img2, border=0):
    # img1 and img2 have range [0, 255]
    #img1 = img1.squeeze()
    #img2 = img2.squeeze()
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    h, w = img1.shape[:2]
    img1 = img1[border:h-border, border:w-border]
    img2 = img2[border:h-border, border:w-border]

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))


def calculate_cost(model, dataloader, device):

  model.eval()
  total_psnr = 0
  with torch.no_grad():
    for noisy_images, clean_images in dataloader:
      noisy_images = noisy_images.to(device)
      clean_images = clean_images.to(device)

      denoised_images = model(noisy_images)

      for i in range(denoised_images.shape[0]):
        denoised_img = denoised_images[i].cpu().numpy().transpose(1, 2, 0)
        clean_img = clean_images[i].cpu().numpy().transpose(1, 2, 0)
        total_psnr += calculate_psnr(denoised_img, clean_img)

  average_psnr = total_psnr / len(dataloader.dataset)
  return average_psnr


In [7]:
# -*- coding: utf-8 -*-
import math
import torch
import torch.nn as nn
import numpy as np
from thop import profile
from einops import rearrange
from einops.layers.torch import Rearrange, Reduce
from timm.models.layers import trunc_normal_, DropPath


class WMSA(nn.Module):
    """ Self-attention module in Swin Transformer
    """

    def __init__(self, input_dim, output_dim, head_dim, window_size, type):
        super(WMSA, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.head_dim = head_dim
        self.scale = self.head_dim ** -0.5
        self.n_heads = input_dim//head_dim
        self.window_size = window_size
        self.type=type
        self.embedding_layer = nn.Linear(self.input_dim, 3*self.input_dim, bias=True)

        # TODO recover
        # self.relative_position_params = nn.Parameter(torch.zeros(self.n_heads, 2 * window_size - 1, 2 * window_size -1))
        self.relative_position_params = nn.Parameter(torch.zeros((2 * window_size - 1)*(2 * window_size -1), self.n_heads))

        self.linear = nn.Linear(self.input_dim, self.output_dim)

        trunc_normal_(self.relative_position_params, std=.02)
        self.relative_position_params = torch.nn.Parameter(self.relative_position_params.view(2*window_size-1, 2*window_size-1, self.n_heads).transpose(1,2).transpose(0,1))

    def generate_mask(self, h, w, p, shift):
        """ generating the mask of SW-MSA
        Args:
            shift: shift parameters in CyclicShift.
        Returns:
            attn_mask: should be (1 1 w p p),
        """
        # supporting sqaure.
        attn_mask = torch.zeros(h, w, p, p, p, p, dtype=torch.bool, device=self.relative_position_params.device)
        if self.type == 'W':
            return attn_mask

        s = p - shift
        attn_mask[-1, :, :s, :, s:, :] = True
        attn_mask[-1, :, s:, :, :s, :] = True
        attn_mask[:, -1, :, :s, :, s:] = True
        attn_mask[:, -1, :, s:, :, :s] = True
        attn_mask = rearrange(attn_mask, 'w1 w2 p1 p2 p3 p4 -> 1 1 (w1 w2) (p1 p2) (p3 p4)')
        return attn_mask

    def forward(self, x):
        """ Forward pass of Window Multi-head Self-attention module.
        Args:
            x: input tensor with shape of [b h w c];
            attn_mask: attention mask, fill -inf where the value is True;
        Returns:
            output: tensor shape [b h w c]
        """
        if self.type!='W': x = torch.roll(x, shifts=(-(self.window_size//2), -(self.window_size//2)), dims=(1,2))
        x = rearrange(x, 'b (w1 p1) (w2 p2) c -> b w1 w2 p1 p2 c', p1=self.window_size, p2=self.window_size)
        h_windows = x.size(1)
        w_windows = x.size(2)
        # sqaure validation
        # assert h_windows == w_windows

        x = rearrange(x, 'b w1 w2 p1 p2 c -> b (w1 w2) (p1 p2) c', p1=self.window_size, p2=self.window_size)
        qkv = self.embedding_layer(x)
        q, k, v = rearrange(qkv, 'b nw np (threeh c) -> threeh b nw np c', c=self.head_dim).chunk(3, dim=0)
        sim = torch.einsum('hbwpc,hbwqc->hbwpq', q, k) * self.scale
        # Adding learnable relative embedding
        sim = sim + rearrange(self.relative_embedding(), 'h p q -> h 1 1 p q')
        # Using Attn Mask to distinguish different subwindows.
        if self.type != 'W':
            attn_mask = self.generate_mask(h_windows, w_windows, self.window_size, shift=self.window_size//2)
            sim = sim.masked_fill_(attn_mask, float("-inf"))

        probs = nn.functional.softmax(sim, dim=-1)
        output = torch.einsum('hbwij,hbwjc->hbwic', probs, v)
        output = rearrange(output, 'h b w p c -> b w p (h c)')
        output = self.linear(output)
        output = rearrange(output, 'b (w1 w2) (p1 p2) c -> b (w1 p1) (w2 p2) c', w1=h_windows, p1=self.window_size)

        if self.type!='W': output = torch.roll(output, shifts=(self.window_size//2, self.window_size//2), dims=(1,2))
        return output

    def relative_embedding(self):
        cord = torch.tensor(np.array([[i, j] for i in range(self.window_size) for j in range(self.window_size)]))
        relation = cord[:, None, :] - cord[None, :, :] + self.window_size -1
        # negative is allowed
        return self.relative_position_params[:, relation[:,:,0].long(), relation[:,:,1].long()]


class Block(nn.Module):
    def __init__(self, input_dim, output_dim, head_dim, window_size, drop_path, type='W', input_resolution=None):
        """ SwinTransformer Block
        """
        super(Block, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        assert type in ['W', 'SW']
        self.type = type
        if input_resolution <= window_size:
            self.type = 'W'

        print("Block Initial Type: {}, drop_path_rate:{:.6f}".format(self.type, drop_path))
        self.ln1 = nn.LayerNorm(input_dim)
        self.msa = WMSA(input_dim, input_dim, head_dim, window_size, self.type)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.ln2 = nn.LayerNorm(input_dim)
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 4 * input_dim),
            nn.GELU(),
            nn.Linear(4 * input_dim, output_dim),
        )

    def forward(self, x):
        x = x + self.drop_path(self.msa(self.ln1(x)))
        x = x + self.drop_path(self.mlp(self.ln2(x)))
        return x


class ConvTransBlock(nn.Module):
    def __init__(self, conv_dim, trans_dim, head_dim, window_size, drop_path, type='W', input_resolution=None):
        """ SwinTransformer and Conv Block
        """
        super(ConvTransBlock, self).__init__()
        self.conv_dim = conv_dim
        self.trans_dim = trans_dim
        self.head_dim = head_dim
        self.window_size = window_size
        self.drop_path = drop_path
        self.type = type
        self.input_resolution = input_resolution

        assert self.type in ['W', 'SW']
        if self.input_resolution <= self.window_size:
            self.type = 'W'

        self.trans_block = Block(self.trans_dim, self.trans_dim, self.head_dim, self.window_size, self.drop_path, self.type, self.input_resolution)
        self.conv1_1 = nn.Conv2d(self.conv_dim+self.trans_dim, self.conv_dim+self.trans_dim, 1, 1, 0, bias=True)
        self.conv1_2 = nn.Conv2d(self.conv_dim+self.trans_dim, self.conv_dim+self.trans_dim, 1, 1, 0, bias=True)

        self.conv_block = nn.Sequential(
                nn.Conv2d(self.conv_dim, self.conv_dim, 3, 1, 1, bias=False),
                nn.ReLU(True),
                nn.Conv2d(self.conv_dim, self.conv_dim, 3, 1, 1, bias=False)
                )

    def forward(self, x):
        conv_x, trans_x = torch.split(self.conv1_1(x), (self.conv_dim, self.trans_dim), dim=1)
        conv_x = self.conv_block(conv_x) + conv_x
        trans_x = Rearrange('b c h w -> b h w c')(trans_x)
        trans_x = self.trans_block(trans_x)
        trans_x = Rearrange('b h w c -> b c h w')(trans_x)
        res = self.conv1_2(torch.cat((conv_x, trans_x), dim=1))
        x = x + res

        return x


class SCUNet(nn.Module):

    def __init__(self, in_nc=3, config=[2,2,2,2,2,2,2], dim=64, drop_path_rate=0.0, input_resolution=256):
        super(SCUNet, self).__init__()
        self.config = config
        self.dim = dim
        self.head_dim = 32
        self.window_size = 8

        # drop path rate for each layer
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(config))]

        self.m_head = [nn.Conv2d(in_nc, dim, 3, 1, 1, bias=False)]

        begin = 0
        self.m_down1 = [ConvTransBlock(dim//2, dim//2, self.head_dim, self.window_size, dpr[i+begin], 'W' if not i%2 else 'SW', input_resolution)
                      for i in range(config[0])] + \
                      [nn.Conv2d(dim, 2*dim, 2, 2, 0, bias=False)]

        begin += config[0]
        self.m_down2 = [ConvTransBlock(dim, dim, self.head_dim, self.window_size, dpr[i+begin], 'W' if not i%2 else 'SW', input_resolution//2)
                      for i in range(config[1])] + \
                      [nn.Conv2d(2*dim, 4*dim, 2, 2, 0, bias=False)]

        begin += config[1]
        self.m_down3 = [ConvTransBlock(2*dim, 2*dim, self.head_dim, self.window_size, dpr[i+begin], 'W' if not i%2 else 'SW',input_resolution//4)
                      for i in range(config[2])] + \
                      [nn.Conv2d(4*dim, 8*dim, 2, 2, 0, bias=False)]

        begin += config[2]
        self.m_body = [ConvTransBlock(4*dim, 4*dim, self.head_dim, self.window_size, dpr[i+begin], 'W' if not i%2 else 'SW', input_resolution//8)
                    for i in range(config[3])]

        begin += config[3]
        self.m_up3 = [nn.ConvTranspose2d(8*dim, 4*dim, 2, 2, 0, bias=False),] + \
                      [ConvTransBlock(2*dim, 2*dim, self.head_dim, self.window_size, dpr[i+begin], 'W' if not i%2 else 'SW',input_resolution//4)
                      for i in range(config[4])]

        begin += config[4]
        self.m_up2 = [nn.ConvTranspose2d(4*dim, 2*dim, 2, 2, 0, bias=False),] + \
                      [ConvTransBlock(dim, dim, self.head_dim, self.window_size, dpr[i+begin], 'W' if not i%2 else 'SW', input_resolution//2)
                      for i in range(config[5])]

        begin += config[5]
        self.m_up1 = [nn.ConvTranspose2d(2*dim, dim, 2, 2, 0, bias=False),] + \
                    [ConvTransBlock(dim//2, dim//2, self.head_dim, self.window_size, dpr[i+begin], 'W' if not i%2 else 'SW', input_resolution)
                      for i in range(config[6])]

        self.m_tail = [nn.Conv2d(dim, in_nc, 3, 1, 1, bias=False)]

        self.m_head = nn.Sequential(*self.m_head)
        self.m_down1 = nn.Sequential(*self.m_down1)
        self.m_down2 = nn.Sequential(*self.m_down2)
        self.m_down3 = nn.Sequential(*self.m_down3)
        self.m_body = nn.Sequential(*self.m_body)
        self.m_up3 = nn.Sequential(*self.m_up3)
        self.m_up2 = nn.Sequential(*self.m_up2)
        self.m_up1 = nn.Sequential(*self.m_up1)
        self.m_tail = nn.Sequential(*self.m_tail)
        #self.apply(self._init_weights)

    def forward(self, x0):

        h, w = x0.size()[-2:]
        paddingBottom = int(np.ceil(h/64)*64-h)
        paddingRight = int(np.ceil(w/64)*64-w)
        x0 = nn.ReplicationPad2d((0, paddingRight, 0, paddingBottom))(x0)

        x1 = self.m_head(x0)
        x2 = self.m_down1(x1)
        x3 = self.m_down2(x2)
        x4 = self.m_down3(x3)
        x = self.m_body(x4)
        x = self.m_up3(x+x4)
        x = self.m_up2(x+x3)
        x = self.m_up1(x+x2)
        x = self.m_tail(x+x1)

        x = x[..., :h, :w]

        return x


    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)



if __name__ == '__main__':

    # torch.cuda.empty_cache()
    net = SCUNet()

    x = torch.randn((7, 3, 64, 128))
    x = net(x)
    print(x.shape)

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
torch.Size([7, 3, 64, 128])


In [ ]:
###Supprimer checkpoint

import os
import shutil

# Spécifiez le chemin du dossier que vous souhaitez explorer
directory_path = './BSD400'  # Remplacez par le chemin de votre dossier

# Lister les fichiers et dossiers du dossier
files = os.listdir(directory_path)

# Supprimer uniquement les dossiers cachés
print("Suppression des dossiers cachés dans le dossier :")
for file in files:
    file_path = os.path.join(directory_path, file)

    # Vérifier si c'est un dossier caché (commence par un point)
    if file.startswith('.') and os.path.isdir(file_path):
        shutil.rmtree(file_path)  # Supprimer le dossier et son contenu
        print(f"Dossier supprimé : {file}")


Suppression des dossiers cachés dans le dossier :


In [ ]:
# Fonction d'entraînement sans evaluer sur le val


def train_scunet(model, dataloader, epochs=10, lr=0.001, device='cuda'):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    #criterion = calculate_cost()

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0

        for noisy_imgs, clean_imgs in dataloader:
          noisy_imgs=noisy_imgs.float()
          clean_imgs=clean_imgs.float()
          print(noisy_imgs.dtype)
          print(clean_imgs.dtype)

          noisy_imgs, clean_imgs = noisy_imgs.to(device), clean_imgs.to(device)

          # Forward pass
          #noisy_imgs=noisy_imgs.double()
          noisy_imgs = noisy_imgs.permute(0, 3, 1, 2)
          clean_imgs = clean_imgs.permute(0, 3, 1, 2)

          outputs = model(noisy_imgs)
          loss = criterion(outputs, clean_imgs)
          #loss=criterion(model,dataloader,device)

          # Backward pass
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          epoch_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(dataloader):.4f}")

# Initialiser et entraîner le modèle
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SCUNet()
train_scunet(model, train_dataloader, epochs=10, lr=0.001, device=device)


Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
torch.float32
torch.float32


: 

In [ ]:
"""
Utilser cette fonction si il y a un val set
"""

def train_1epoch(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)

    # Set the model to training mode.
    # It is important for layers behaving differently between
    # training and evaluation, such as batch normalization and dropout layers.
    # Unnecessary in this situation but added for the best practice.
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)  # Model and data must be on the same device!
        y = y.to(device)

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Display loss from time to time
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss: >7f}  [{current: >5d} / {size: >5d}]")


def evaluate(dataloader, model, loss_fn, device, set_='Validation'):
    # Set the model to evaluation mode.
    # It is important for layers behaving differently between
    # training and evaluation, such as batch normalization and dropout layers.
    # Unnecessary in this situation but added for the best practice.
    model.eval()
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            # Move the tensors to `device`. For instance, it could be a GPU.
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"\n{set_} set: ")  # \n means "new line"
    print(f" Aaccuracy: {(100 * correct): >0.1f}%,"
          f" Avg loss: {test_loss: >8f} \n")

In [ ]:
learning_rate = 1e-3
model = SCUNet()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_fn_mean = nn.CrossEntropyLoss(reduction='mean')  # for training
loss_fn_sum = nn.CrossEntropyLoss(reduction='sum')  # for evaluation

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

print(f"Using {device} device")

model.to(device)  # `to` methods changes the state for torch.nn.Module.
                  # We do not need to reassign the variable.

Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Using cpu device


SCUNet(
  (m_head): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (m_down1): Sequential(
    (0): ConvTransBlock(
      (trans_block): Block(
        (ln1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (msa): WMSA(
          (embedding_layer): Linear(in_features=32, out_features=96, bias=True)
          (linear): Linear(in_features=32, out_features=32, bias=True)
        )
        (drop_path): Identity()
        (ln2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=32, out_features=128, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=128, out_features=32, bias=True)
        )
      )
      (conv1_1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (conv1_2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (conv_block): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), pa

In [ ]:
epochs = 10

for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train_1epoch(train_dataloader, model, loss_fn_mean, optimizer, device)
    #evaluate(validation_dataloader, model, loss_fn_sum, device)

print("Done!")

### CHARGEMENT MODELE

In [ ]:
model = SCUNet()
model.load_state_dict(torch.load('model_weights_epoch_10.pth', map_location=torch.device('cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model =model.double()
model.to(device)

In [24]:
elemdata=[]
for noisy_images, clean_images in train_dataloader:
    noisy_images=noisy_images.permute(0, 3, 1, 2)
    noisy_images=noisy_images.double()
    clean_images=clean_images.double()
    clean_images=clean_images.permute(0, 3, 1, 2)
    elemdata.append((noisy_images))
    print(noisy_images.dtype)
    print(clean_images.shape)
    #break
len(elemdata)

def denormalize_images(images):
    return images * 255.0

torch.float64
torch.Size([64, 3, 128, 128])
torch.float64
torch.Size([64, 3, 128, 128])
torch.float64
torch.Size([64, 3, 128, 128])
torch.float64
torch.Size([64, 3, 128, 128])
torch.float64
torch.Size([64, 3, 128, 128])
torch.float64
torch.Size([64, 3, 128, 128])
torch.float64
torch.Size([16, 3, 128, 128])


In [ ]:
###Execute uniquement ceci pour l'instant

for noisy_images in elemdata:
    noisy_images = noisy_images.to(device)
    outputs = model(noisy_images)
    outputs = outputs.to('cuda')
    outputs = denormalize_images(outputs)
    outputs = outputs.detach().numpy()
    outputs = outputs.astype(np.uint8)
    outputs = np.transpose(outputs, (0, 2, 3, 1))
    for i in range(outputs.shape[0]):
        plt.imshow(outputs[i])
        plt.show()
        psnr=calculate_psnr(outputs[i],clean_images[i])
        print(psnr)
        break

In [ ]:
#model.eval()  # Passer en mode évaluation

img_noisy_tensor = test[0].to(device)  # Déplacer l'image bruitée sur le bon appareil
model.double()
with torch.no_grad():
    print(img_noisy_tensor.dtype)
    #img_noisy_tensor = test[0].to(device).double()
    #img_noisy_tensor = img_noisy_tensor.permute(2, 0, 1).unsqueeze(0)
    denoised_image = model(img_noisy_tensor)

In [ ]:
import matplotlib.pyplot as plt

def visualize_denoising(model, dataloader, device='cpu'):
    model.eval()
    noisy_imgs, clean_imgs = next(iter(dataloader))
    noisy_imgs=noisy_imgs.permute(0, 3, 1, 2)
    clean_imgs=clean_imgs.permute(0, 3, 1, 2)

    noisy_imgs, clean_imgs = noisy_imgs.to(device), clean_imgs.to(device)

    with torch.no_grad():
        denoised_imgs = model(noisy_imgs)

    # Affichage
    fig, axs = plt.subplots(3, 5, figsize=(15, 9))
    for i in range(5):
        axs[0, i].imshow(noisy_imgs[i].cpu().squeeze(), cmap='gray')
        axs[0, i].set_title("Noisy")
        axs[1, i].imshow(denoised_imgs[i].cpu().squeeze(), cmap='gray')
        axs[1, i].set_title("Denoised")
        axs[2, i].imshow(clean_imgs[i].cpu().squeeze(), cmap='gray')
        axs[2, i].set_title("Clean")
    plt.tight_layout()
    plt.show()

# Visualiser
model.double()
visualize_denoising(model, train_dataloader, device=device)
